# Basic Analysis

We are using KeplerGL
```
jupyter nbextension install --py --sys-prefix keplergl # can be skipped for notebook 5.3 and above
jupyter nbextension enable --py --sys-prefix keplergl # can be skipped for notebook 5.3 and above
```

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import geopandas as gpd
import pathlib
import sqlite3
import pandas as pd

data_dir = pathlib.Path("..").resolve() / "data"
db_con = sqlite3.connect(str(data_dir / "ppm.gpkg"))
# Create connection and load spatialite extension
df = pd.read_sql('SELECT * FROM "ministers" AS "a" JOIN "education" AS "b" ON ("a"."school name" = "b"."school_name");', db_con)
# df = gpd.read_postgis('SELECT * FROM "ministers" AS "a" JOIN "education" AS "b" ON ("a"."school name" = "b"."school_name");', db_con, geom_col="geom")
geometry = gpd.GeoSeries.from_wkt(df.wkt)
df["Longitude"] =geometry.x
df["Latitude" ] =geometry.y
# gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=4326)
# gdf.drop(columns=["geom"], inplace=True)
df.drop(columns=["geom", "wkt"], inplace=True)

In [3]:
from keplergl import KeplerGl
map = KeplerGl(height=1600)
map.add_data(data=df, name="schools")
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/keplergl/keplergl.py:22: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df.to_dict('split')
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/jupyter_client/session.py:719: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'schools': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

In [4]:
df_missing_high_school = pd.read_sql('SELECT * FROM "ministers" AS "a" LEFT OUTER JOIN (SELECT * FROM "education" WHERE "is_high_school" = 1) AS "b" ON ("a"."school name" = "b"."school_name") ;', db_con)

In [5]:
members_with_no_hs = df_missing_high_school[~df_missing_high_school.member.isin(df_missing_high_school [df_missing_high_school.is_high_school > 0]["member"])].sort_values(by=["member"])

In [13]:
member_mps = members_with_no_hs.drop_duplicates(["member", "wiki link" , "mp_ids"])[["member", "wiki link" , "mp_ids"]]
member_mps

,member,wiki link,mp_ids
356,Andrew McLachlan,http://www.wikidata.org/entity/Q16732352,287062
141,Andrew Wallace,http://www.wikidata.org/entity/Q25756194,265967
174,Anne Webster,http://www.wikidata.org/entity/Q64216519,281688
415,Bridget McKenzie,http://www.wikidata.org/entity/Q4966734,207825
220,Carina Garland,http://www.wikidata.org/entity/Q112108318,295588
...,...,...,...
80,Warren Entsch,http://www.wikidata.org/entity/Q7970178,7K6
326,Wendy Askew,http://www.wikidata.org/entity/Q61983633,281558
224,Zaneta Mascarenhas,http://www.wikidata.org/entity/Q112106576,298800
241,Zoe Daniel,http://www.wikidata.org/entity/Q65628647,008CH


In [15]:
import requests
import urllib3
urllib3.disable_warnings()

PARLIMENTARIAN_HANDBOOK = "https://handbookapi.aph.gov.au/api/individuals?$orderby=FamilyName,GivenName &$filter=PHID eq '{ph_id}'"
def parlimentarian_handbook_secondary_school(ph_id):
    r = requests.get(PARLIMENTARIAN_HANDBOOK.format(ph_id=ph_id), verify=False)
    r.raise_for_status()
    query = r.json()
    values = query.get("value", [])
    if len(values):
        return ",".join([value.get("SecondarySchool") for value in values])

In [16]:
member_mps["school_names"] = member_mps.mp_ids.apply(parlimentarian_handbook_secondary_school).str.split("/")

In [26]:
member_mps["no_schools"] = member_mps.school_names.map(lambda d: len([ _ for _ in d if _]))

In [30]:
members_to_update = member_mps[member_mps["no_schools"]> 0].explode("school_names")[["member","school_names"]]

In [37]:
# Add mp id to all ministers
import pathlib
import sqlite3
import pandas as pd

data_dir = pathlib.Path("..").resolve() / "data"
db_con = sqlite3.connect(str(data_dir / "ppm.gpkg"))
# Create connection and load spatialite extension
df = pd.read_sql('SELECT * FROM "ministers"', db_con)
df.drop_duplicates("member", inplace=True)
ministers = df.merge(members_to_update)
# ministers.to_sql("ministers", db_con, if_exists="replace", index=False)


In [38]:
ministers["school name"] = ministers["school_names"]

In [39]:
ministers.drop(columns="school_names", inplace=True)

In [40]:
ministers.to_sql("ministers", db_con, if_exists="append", index=False)

84

In [41]:
# update missing schools
import pathlib
import sqlite3
import geopandas as gpd

data_dir = pathlib.Path("..").resolve() / "data"
gpkg = str(data_dir / "ppm.gpkg")
# Create connection and load spatialite extension
df = gpd.read_file(gpkg, layer="education")

In [46]:
schools_to_append = gpd.GeoDataFrame(members_to_update[~members_to_update["school_names"].isin(df.school_name)]["school_names"])

In [49]:
schools_to_append.rename(columns={"school_names": "school_name"}, inplace=True)

In [51]:
schools_to_append["is_university"]=0
schools_to_append["is_high_school"]=1
schools_to_append["is_alt_edu"]=0
df.append(schools_to_append)

/tmp/ipykernel_81481/3504980194.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(schools_to_append)
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(


,ogc_fid,school_name,school_link,is_university,is_high_school,is_alt_edu,operational_status,wkt,is_public,geometry
0,1.0,New College,http://www.wikidata.org/entity/Q1376987,1,0,0,NaN,Point(-1.25166667 51.75416667),NaN,POINT (-1.25167 51.75417)
1,2.0,"Mazenod College, Perth",http://www.wikidata.org/entity/Q1914698,0,1,0,NaN,Point(116.039 -32.0067),NaN,POINT (116.03900 -32.00670)
2,3.0,Royal Military College,http://www.wikidata.org/entity/Q2171074,0,0,1,NaN,Point(149.165 -35.299722222),NaN,POINT (149.16500 -35.29972)
3,4.0,Anglican Church Grammar School,http://www.wikidata.org/entity/Q4763439,0,1,0,NaN,Point(153.054 -27.4819),NaN,POINT (153.05400 -27.48190)
4,5.0,Aquinas College,http://www.wikidata.org/entity/Q4782971,0,1,0,NaN,Point(145.235555555 -37.823055555),NaN,POINT (145.23556 -37.82306)
...,...,...,...,...,...,...,...,...,...,...
224,NaN,Kambalda Senior High School,NaN,0,1,0,NaN,NaN,NaN,None
224,NaN,John Paul College,NaN,0,1,0,NaN,NaN,NaN,None
241,NaN,Queechy High School (Launceston),NaN,0,1,0,NaN,NaN,NaN,None
241,NaN,Rosny College (Hobart),NaN,0,1,0,NaN,NaN,NaN,None
